In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup

# Function to extract domains from dynamic content using Selenium
def extract_dynamic_domains(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//script | //link | //img | //iframe")))

        domains = set()
        for tag in driver.find_elements(By.XPATH, "//script | //link | //img | //iframe"):
            src = tag.get_attribute("src") or tag.get_attribute("href")
            if src:
                domain = urlparse(src).netloc
                if domain:
                    domains.add(domain)

        return domains
    finally:
        driver.quit()

# Function to extract domains from static content using BeautifulSoup
def extract_static_domains(website_url):
    try:
        response = requests.get(website_url, timeout=10)
        response.raise_for_status()
        parsed_url = urlparse(website_url).netloc

        soup = BeautifulSoup(response.text, 'html.parser')
        domains = set()

        for tag in soup.find_all(['script', 'link', 'img', 'iframe']):
            url = tag.get('src') or tag.get('href')
            if url:
                domain = urlparse(url).netloc
                if domain and domain != parsed_url:
                    domains.add(domain)

        return domains
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {website_url}: {e}")
        return set()

# Function to generate commands
def generate_commands(num_urls, websites):
    all_domains = {}
    for website in websites:
        dynamic_domains = extract_dynamic_domains(website)
        static_domains = extract_static_domains(website)
        all_domains[website] = dynamic_domains.union(static_domains)

    combined_domains = set()
    for website in websites:
        combined_domains.add(urlparse(website).netloc)
    for domains in all_domains.values():
        combined_domains.update(domains)

    first_url = websites[0]
    parsed_first_url = urlparse(first_url)
    first_domain = parsed_first_url.netloc
    wd_path = parsed_first_url.path

    teltonika_command = f"""1. COMANDO TELTONIKA
uci set chilli.@chilli[0].uamserver="{first_url}"
uci commit
path=$(uci get chilli.@chilli[0].uamdomainfile)
cat /dev/null > $path
"""
    for domain in sorted(combined_domains):
        teltonika_command += f"echo -e \"{domain}\" >> $path\n"
    teltonika_command += "/etc/init.d/chilli restart\n"

    four_faith_command = f"""2. COMANDO FOUR FAITH
nvram set qq_domain_list "{' '.join(sorted(combined_domains))}" && \
nvram set wd_hostname {first_domain} && \
nvram set wd_path {wd_path} && \
nvram commit && \
reboot
"""

    return teltonika_command, four_faith_command

# Create widgets for input
num_urls_widget = widgets.IntSlider(value=1, min=1, max=4, description="Number of URLs:", continuous_update=False)
url_widgets = [widgets.Text(placeholder="Enter URL", description=f"URL {i+1}:") for i in range(4)]

# Output text boxes for commands
teltonika_output = widgets.Textarea(
    description="Teltonika Command:",
    layout=widgets.Layout(width='50%', height='300px')
)

four_faith_output = widgets.Textarea(
    description="Four Faith Command:",
    layout=widgets.Layout(width='50%', height='300px')
)

# Function to handle button click event
def on_button_click(b):
    # Get the number of URLs and the list of URLs
    num_urls = num_urls_widget.value
    websites = [url_widgets[i].value.strip() for i in range(num_urls)]

    # Validate URLs
    invalid_urls = [url for url in websites if not url.startswith('http')]
    if invalid_urls:
        teltonika_output.value = f"Invalid URLs: {', '.join(invalid_urls)}"
        four_faith_output.value = ""
        return  # Exit if there are invalid URLs

    # Generate the commands
    teltonika_command, four_faith_command = generate_commands(num_urls, websites)

    # Update the output text areas with the generated commands
    teltonika_output.value = teltonika_command
    four_faith_output.value = four_faith_command

    # Optionally, save the commands to a file
    with open("generated_commands.sh", "w") as file:
        file.write(teltonika_command + "\n" + four_faith_command)
    print("\nCommands saved to generated_commands.sh")

# Create a button to trigger command generation
generate_button = widgets.Button(description="Generate Commands")
generate_button.on_click(on_button_click)

# Create layout
input_box = widgets.VBox([num_urls_widget] + url_widgets + [generate_button])
output_box = widgets.HBox([teltonika_output, four_faith_output])

# Display the interface with input widgets on the left and outputs on the right
display(input_box, output_box)

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 25.8 MB/s eta 0:00:00


In [ ]:
!pip install gspread oauth2client

In [ ]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.4 MB/s eta 0:00:00


In [ ]:
!pip install flask-ngrok